In [4]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
ca_aq_df = pd.read_csv('cleaned_data/ca_aq_data.csv')
il_aq_df = pd.read_csv('cleaned_data/il_aq_data.csv')
ny_aq_df = pd.read_csv('cleaned_data/ny_aq_data.csv')
ga_aq_df = pd.read_csv('cleaned_data/ga_aq_data.csv')

In [7]:
ca_aq_mean = ca_aq_df['arithmetic_mean'].mean()
il_aq_mean = il_aq_df['arithmetic_mean'].mean()
ny_aq_mean = ny_aq_df['arithmetic_mean'].mean()
ga_aq_mean = ga_aq_df['arithmetic_mean'].mean()

In [9]:
ca_cancer_rates = pd.read_csv('cleaned_data/ca_all_counties_cancer_rates.csv')
il_cancer_rates = pd.read_csv('cleaned_data/il_all_counties_cancer_rates.csv')
ny_cancer_rates = pd.read_csv('cleaned_data/ny_all_counties_cancer_rates.csv')
ga_cancer_rates = pd.read_csv('cleaned_data/ga_all_counties_cancer_rates.csv')

In [10]:
ca_cancer_rates.head()

,Unnamed: 0,County,Year,AgeAdjustedRate,CaseCount,Population
0,0,Mono,2012-2016,25.9,20,70372
1,1,Marin,2012-2016,35.1,680,1299494
2,2,Monterey,2012-2016,36.8,763,2154029
3,3,Los Angeles,2012-2016,36.6,18307,50321257
4,4,Santa Cruz,2012-2016,37.0,538,1357213


In [11]:
cancer_rates = [ca_cancer_rates, il_cancer_rates, ny_cancer_rates, ga_cancer_rates]
for i in cancer_rates:
    i.drop(columns=['Unnamed: 0'], inplace=True)

In [12]:
ca_cancer_rates.head()

,County,Year,AgeAdjustedRate,CaseCount,Population
0,Mono,2012-2016,25.9,20,70372
1,Marin,2012-2016,35.1,680,1299494
2,Monterey,2012-2016,36.8,763,2154029
3,Los Angeles,2012-2016,36.6,18307,50321257
4,Santa Cruz,2012-2016,37.0,538,1357213


In [13]:
ca_cancer_rate = list(ca_cancer_rates['AgeAdjustedRate'])

In [16]:
ca_aq_ozone = list(ca_aq_df.loc[ca_aq_df.parameter == 'Ozone', 'arithmetic_mean'])

In [20]:
stats.ttest_ind(ca_aq_ozone, ca_cancer_rate, equal_var=False)

Ttest_indResult(statistic=-40.842553528287645, pvalue=7.905230952890071e-43)